In [231]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

#### melon webpage 연결하기

In [4]:
ser = Service('../chromedriver/chromedriver.exe')

driver = webdriver.Chrome(service = ser)

In [5]:
url = 'https://www.melon.com/chart/index.htm'
driver.get(url)

In [8]:
html = driver.page_source
html

soup = BeautifulSoup(html, 'html.parser')

#### html 구조 파악하기

In [9]:
# 예제 2-39 HTML 구조
html = '''
<html>
    <head>
    </head>
    <body>
        <h1> 우리동네시장</h1>
            <div class = 'sale'>
                <p id='fruits1' class='fruits'>
                    <span class = 'name'> 바나나 </span>
                    <span class = 'price'> 3000원 </span>
                    <span class = 'inventory'> 500개 </span>
                    <span class = 'store'> 가나다상회 </span>
                    <a href = 'http://bit.ly/forPlaywithData' > 홈페이지 </a>
                </p>
            </div>
            <div class = 'prepare'>
                <p id='fruits2' class='fruits'>
                    <span class ='name'> 파인애플 </span>
                </p>
            </div>
    </body>
</html>
'''

In [13]:
soup = BeautifulSoup(html, 'html.parser')

In [21]:
tags_span = soup.select('span')
tags_span[0].text

' 바나나 '

In [23]:
tags_p = soup.select('p')
len(tags_p)

2

In [31]:
class_name = soup.select('.name')

class_name_text = [ class_name[i].text for i in range(0,len(class_name))]
class_name_text

[' 바나나 ', ' 파인애플 ']

In [37]:
soup.select('span.name')
soup.select('span.inventory')
soup.select('#fruits1 > span.name')

[<span class="name"> 바나나 </span>]

In [38]:
soup.select('div.sale span.store')

[<span class="store"> 가나다상회 </span>]

In [51]:
tags_a = soup.select('a')

tag = tags_a[0]
print(tag.text.strip())
print(tag['href'])

홈페이지
http://bit.ly/forPlaywithData


#### melon top100차트 분석하기

In [53]:
url = 'https://www.melon.com/chart/index.htm'

driver.get(url)

In [165]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

#songs = soup.select('tr')     
#song_list = songs[1:]            # top100에 불포함되는 0[첫번째]행 제외하고 song에 저장 
                                  # >> 이방법보다는 아래의 방법 추천(명확한 경로 입력)

#### 교재

In [168]:
songs = soup.select('table > tbody > tr')

for song in songs:
    title = song.select('div.ellipsis.rank01 > span > a')[0].text
    singer = song.select ('div.ellipsis.rank02 > a')[0].text  
    #print(title, singer, sep = ' | ')

##### 내가 작성한 코드 - 1

In [192]:
# 순위, 제목, 가수
songs = soup.select('tbody > tr')
song_data = []

for i in range(len(songs)):
    song = songs[i].select('.wrap_song_info > div')
    song_name = song[0].text.strip('\n')
    singer = song[1].select('a')[0].text
    #print(i+1,'.', song_name[i], '-' ,singer[i])
    song_data.append(['melon', i+1, song_name, singer])

song_data_df = DataFrame(song_data, columns = ['서비스','순위','타이틀', '가수'])
song_data_df

,서비스,순위,타이틀,가수
0,melon,1,취중고백,김민석 (멜로망스)
1,melon,2,사랑은 늘 도망가,임영웅
2,melon,3,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo
3,melon,4,Counting Stars (Feat. Beenzino),BE'O (비오)
4,melon,5,겨울잠,아이유
...,...,...,...,...
95,melon,96,ASAP,STAYC(스테이씨)
96,melon,97,"호우주의 (Feat. 개코, 넉살 (Nucksal)) (Prod. 코드 쿤스트)",조광일
97,melon,98,19금\nPeaches (Feat. Daniel Caesar & Giveon),Justin Bieber
98,melon,99,밤새 서로 미루다,존박


#### 크롤링결과 저장하기

In [189]:
songs = soup.select('tbody > tr')
song_data2 = []
rank = 1

for song in songs:
    title = song.select('div.ellipsis.rank01 > span > a')[0].text
    singer = song.select ('div.ellipsis.rank02 > a')[0].text  
    #print(title, singer, sep = ' | ')
    song_data2.append(['melon', rank, title, singer])
    rank += 1

In [190]:
song_data2_df = DataFrame(song_data2, columns = ['서비스','순위','타이틀', '가수'])
song_data2_df

,서비스,순위,타이틀,가수
0,melon,1,취중고백,김민석 (멜로망스)
1,melon,2,사랑은 늘 도망가,임영웅
2,melon,3,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo
3,melon,4,Counting Stars (Feat. Beenzino),BE'O (비오)
4,melon,5,겨울잠,아이유
...,...,...,...,...
95,melon,96,ASAP,STAYC(스테이씨)
96,melon,97,"호우주의 (Feat. 개코, 넉살 (Nucksal)) (Prod. 코드 쿤스트)",조광일
97,melon,98,Peaches (Feat. Daniel Caesar & Giveon),Justin Bieber
98,melon,99,밤새 서로 미루다,존박


In [191]:
song_data2_df.to_excel('./files/melon_class.xlsx', 
                   index = False,
                   sheet_name = 'top100')

#### 벅스 top100 크롤링하기

In [193]:
url = 'https://music.bugs.co.kr/chart'

driver.get(url)

In [197]:
html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

#### 내가 작성한 코드 - 벅스 1

In [306]:
title_soup = soup.select('tbody > tr p.title')
singer_soup = soup.select('tbody > tr p.artist')
bugs_song = []

for i in range(0,100) : 
    song_name = title_soup[i].text.replace('\n','')
    singer = singer_soup[i].text.replace('\n','')
    bugs_song.append(['bugs', i+1, song_name, singer])
    
bugs_song_df = DataFrame(bugs_song, columns = ['서비스','순위','타이틀', '가수'])
bugs_song_df

,서비스,순위,타이틀,가수
0,bugs,1,Step Back,GOT the beat
1,bugs,2,여름비,샘김(Sam Kim)
2,bugs,3,Christmas Tree,V
3,bugs,4,서랍,10CM
4,bugs,5,Dreams Come True,aespa
...,...,...,...,...
95,bugs,96,사랑은 말로 표현하는게 아니래요,규현 (KYUHYUN)
96,bugs,97,RE : WIND,이세계 아이돌
97,bugs,98,Love Yourself,Justin Bieber(저스틴 비버)
98,bugs,99,"Hey Mama (feat. Nicki Minaj, Bebe Rexha & Afro...",David Guetta(데이빗 게타)


#### 내가 작성한 코드 - 벅스 2

In [300]:
song = soup.select('tbody > tr p')
bugs_song = []

for i in range(0,len(song),3) : 
    song_name = song[i+1].text.replace('\n','')
    singer = song[i+2].text.replace('\n','')
    bugs_song.append(['bugs', int(i/3+1), song_name, singer])
    
bugs_song_df = DataFrame(bugs_song, columns = ['서비스','순위','타이틀', '가수'])
bugs_song_df

,서비스,순위,타이틀,가수
0,bugs,1,Step Back,GOT the beat
1,bugs,2,여름비,샘김(Sam Kim)
2,bugs,3,Christmas Tree,V
3,bugs,4,서랍,10CM
4,bugs,5,Dreams Come True,aespa
...,...,...,...,...
95,bugs,96,사랑은 말로 표현하는게 아니래요,규현 (KYUHYUN)
96,bugs,97,RE : WIND,이세계 아이돌
97,bugs,98,Love Yourself,Justin Bieber(저스틴 비버)
98,bugs,99,"Hey Mama (feat. Nicki Minaj, Bebe Rexha & Afro...",David Guetta(데이빗 게타)


#### 교재

In [308]:
songs = soup.select('table.byChart > tbody > tr')
song_list = []

for i in range(len(songs)):
    title = songs[i].select('p.title > a')[0].text
    singer = songs[i].select('p.artist > a')[0].text
    song_list.append(['Bugs', i+1, title, singer])

song_list_df = DataFrame(song_list, columns = ['서비스','순위','타이틀', '가수'])
song_list_df.to_excel('./files/bugs.xlsx')

#### 지니차트 가져오기

In [309]:
url = 'https://www.genie.co.kr/chart/top200'
driver.get(url)

In [310]:
html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

#### 내가 작성한 코드 - 지니 1

In [358]:
songs = soup.select('div.newest-list tbody tr')
genie_list = []

for i in range(len(songs)):
    title = songs[i].select('.title')[0].text.strip()
    artist = songs[i].select('.artist')[0].text
    genie_list.append(['genie', i+1, title, artist])
    
genie_list_df = DataFrame(genie_list,columns = ['서비스','순위','타이틀', '가수'])
genie_list_df.to_excel('./files/genie_clas.xlsx')

#### 교재

In [360]:
songs = soup.select('tbody > tr')
song_list = []
rank = 1

for song in songs : 
    title = song.select('a.title')[0].text.strip()
    singer = song.select('a.artist')[0].text
    song_list.append(['지니',rank, title, singer])
    rank += 1

song_list_df = DataFrame(song_list, columns = ['서비스','순위','타이틀', '가수'])
song_list_df.to_excel('./files/genie_class2.xlsx', index=False)

#### 멜론, 벅스, 지니 크롤링 엑셀 파일 통합하기

In [363]:
excel_names = ['./files/melon_class.xlsx',
               './files/bugs.xlsx',
               './files/genie_class2.xlsx']

append_data = pd.DataFrame()

for name in excel_names:
    pd_data = pd.read_excel(name)
    append_data = append_data.append(pd_data)
    
append_data.to_excel('./files/append_data.xlsx', index = False)